In [1]:
import os
import pandas as pd
import numpy as np
import nibabel as nib 
import matplotlib.pyplot as plt
import SUITPy.flatmap as flatmap 
from matplotlib.ticker import FormatStrFormatter

In [53]:
workdir = '/data/tu_gaor/scripts'
homedir = '/Users/ruohan/My_Repos/Cerebellum'
cwdir   = homedir
data_dir = os.path.join(cwdir, 'dat')

code_dir = os.path.join(cwdir, 'plotting')
# Dataset and DV of interest
dataset_name = "hcpd"

plot_dir = os.path.join(data_dir, dataset_name, 'from_gam/plots')

# Plot R2 data
- load data and tranpose to rowwise for plotting
- set the negative R2 to 0
- load the cerebeller altlas and reorder the data accordingly
- check data range to decide range for color bar between plots
- function for plotting
- plot and save the .nii files and flatmap plots

In [54]:
fusion_r2 = pd.read_csv(os.path.join(data_dir, dataset_name, 'from_gam', f"{dataset_name}_GAM_r2_fusion.csv"))
print(fusion_r2.columns)

Index(['region', 'r2_age', 'r2_sex', 'r2_srsmoo', 'r2_srage1', 'r2_srage2',
       'r2_srsex', 'r2_lonsmoo', 'r2_lonage1', 'r2_lonsex', 'r2_rejsmoo',
       'r2_rejage1', 'r2_hossmoo', 'r2_hosage1', 'r2_hossex', 'r2_emosmoo',
       'r2_emoage1', 'r2_emosex', 'r2_frismoo', 'r2_friage1', 'r2_frisex'],
      dtype='object')


In [55]:
# transpose the data to plot per row
fusion_r2 = fusion_r2.T
fusion_r2.columns = fusion_r2.iloc[0]
fusion_r2 = fusion_r2[1:]

In [56]:
fusion_r2.head()

region,A1L,A1R,A2L,A2R,A3L,A3R,D1L,D1R,D2L,D2R,...,S1L,S1R,S2L,S2R,S3L,S3R,S4L,S4R,S5L,S5R
r2_age,0.078,0.057,0.016,0.028,0.035,0.062,0.091,0.056,0.044,0.048,...,0.111,0.045,0.078,0.013,0.049,0.02,0.031,0.031,-0.002,0.023
r2_sex,0.004,0.004,0.004,-0.001,0.01,0.002,-0.002,0.002,-0.001,0.002,...,0.005,0.003,-0.002,0.005,0.0,0.008,0.002,0.001,0.01,-0.002
r2_srsmoo,-0.001,0.011,0.002,0.004,0.002,-0.001,-0.01,0.004,0.0,0.003,...,-0.002,0.009,0.001,0.003,-0.001,0.007,0.001,0.002,0.001,-0.002
r2_srage1,-0.003,0.013,0.002,0.005,0.001,-0.002,-0.012,0.004,0.001,0.004,...,-0.003,0.009,0.002,0.005,-0.002,0.007,0.0,0.003,0.001,-0.001
r2_srage2,0.0,0.001,0.0,0.017,0.0,0.004,0.0,0.0,0.0,0.001,...,0.0,0.001,0.001,0.002,0.0,0.0,0.0,0.0,0.0,0.002


In [57]:
# set small negative R² values to 0
data_r2 = fusion_r2.clip(lower=0)

In [58]:
# Create cerebellum template based on fusion atlas
c_path = os.path.join(code_dir, 'atlases/fusion', 'atl-NettekovenSym32_space-MNI_dseg.nii')
cerebellum = nib.load(c_path)
data_c = cerebellum.get_fdata()
region_labels = np.unique(data_c)

In [59]:
# Create a dataframe of region names
lab_num = pd.read_csv(os.path.join(code_dir, 'atlases/fusion', 'atl-NettekovenAsym32.csv'), header=None)  
lab_num.columns = ['parcel_name']
lab_num['parcel'] = range(1, 33)

In [60]:
# Reorder data columns to match the order defined in lab_num
data_r2_ordered = data_r2[lab_num['parcel_name'].tolist()]
print(data_r2_ordered.columns)

Index(['M1L', 'M2L', 'M3L', 'M4L', 'A1L', 'A2L', 'A3L', 'D1L', 'D2L', 'D3L',
       'D4L', 'S1L', 'S2L', 'S3L', 'S4L', 'S5L', 'M1R', 'M2R', 'M3R', 'M4R',
       'A1R', 'A2R', 'A3R', 'D1R', 'D2R', 'D3R', 'D4R', 'S1R', 'S2R', 'S3R',
       'S4R', 'S5R'],
      dtype='object', name='region')


In [61]:
row_min = data_r2_ordered.min(axis=1)
row_max = data_r2_ordered.max(axis=1)
row_range = pd.DataFrame({
    'min': row_min,
    'max': row_max
})
row_range

,min,max
r2_age,0,0.111
r2_sex,0,0.015
r2_srsmoo,0,0.013
r2_srage1,0,0.013
r2_srage2,0.0,0.017
r2_srsex,0,0.021
r2_lonsmoo,0,0.007
r2_lonage1,0,0.007
r2_lonsex,0,0.017
r2_rejsmoo,0,0.003


In [62]:
def style_colorbar(decimals=3):
    """
    Set the colorbar ticks to top and bottom and format labels with fixed decimals.
    
    Parameters
    ----------
    decimals : int
        Number of decimal places to display.
    """
    # Get the current figure's last axis (colorbar)
    cbar = plt.gcf().axes[-1]

    # Set ticks only at min and max
    cbar_min, cbar_max = cbar.get_ylim()
    cbar.set_yticks([cbar_min, cbar_max])

    # Format labels with fixed number of decimals
    fmt = f"{{:.{decimals}f}}"
    cbar.set_yticklabels([fmt.format(cbar_min), fmt.format(cbar_max)])

In [63]:
def make_flatmaps(df, nii_save_dir, plot_save_dir, 
                  symmetric=True, global_range=False, 
                  scale_digits=0, cmap='PuOr_r', 
                  vmin=None, vmax=None):  # <-- define vmin, vmax if needed
    """
    Create and save flatmaps and NIfTI files from a dataframe of region values.

    Parameters
    ----------
    df : pandas.DataFrame
        Each row = one map; columns = region values.
    nii_save_dir : str
        Directory to save .nii.gz files.
    plot_save_dir : str
        Directory to save flatmap PNGs.
    symmetric : bool
        If True, color scale symmetric around zero.
    global_range : bool
        If True, compute color scale across entire df.
        If False, compute color scale per row.
    cmap : str
        Colormap name.
    vmin, vmax : float or None
        If provided, overrides automatic color scaling.
    """
    os.makedirs(nii_save_dir, exist_ok=True)
    os.makedirs(plot_save_dir, exist_ok=True)

    # ---- Compute global color scale if global_range is True ----
    if global_range:
        all_values = df.values.ravel()
        if symmetric:
            abs_max = np.max(np.abs(all_values))
            rounded_max = np.ceil(abs_max * 1000) / 1000
            global_min, global_max = -rounded_max, rounded_max
        else:
            vmin_auto = np.min(all_values)
            vmax_auto = np.max(all_values)
            global_min = np.floor(vmin_auto * 1000) / 1000
            global_max = np.ceil(vmax_auto * 1000) / 1000
        print(f"[INFO] Global color scale: min={global_min}, max={global_max}")
    
    for index, row in df.iterrows():
        new_img = np.zeros_like(data_c)
    
        # Fill cerebellar regions with row values
        for i, region in enumerate(region_labels):
            if region != 0:
                new_img[data_c == region] = row[i - 1]
    
        if np.all(new_img == 0):
            print(f"[SKIP] {index}: no significant values to plot.")
            continue
    
        # ---- Determine color scale ----
        if vmin is not None and vmax is not None:
            # Use manual min/max if provided
            scale_min, scale_max = vmin, vmax
        elif not global_range:
            # Compute per-row color scale
            values = row.values
            if symmetric:
                abs_max = np.max(np.abs(values))
                rounded_max = np.ceil(abs_max * 1000) / 1000
                scale_min, scale_max = -rounded_max, rounded_max
            else:
                vmin_auto = np.min(values)
                vmax_auto = np.max(values)
                scale_min = np.floor(vmin_auto * 1000) / 1000
                scale_max = np.ceil(vmax_auto * 1000) / 1000
        else:
            # Use previously computed global scale
            scale_min, scale_max = global_min, global_max

        print(f"[INFO] {index}: min={scale_min}, max={scale_max}")

        # ---- Save NIfTI ----
        name = str(index)
        img = nib.Nifti1Image(new_img, cerebellum.affine)
        # nii_path = os.path.join(nii_save_dir, f'{dataset_name}_{name}.nii.gz')
        # nib.save(img, nii_path)

        # ---- Generate and save flatmap ----
        c_map = flatmap.vol_to_surf(img)

        ax = flatmap.plot(
            data=c_map,
            cmap=cmap,
            cscale=[scale_min, scale_max],  # <-- use scale_min/scale_max
            render='matplotlib',
            colorbar=True,
            new_figure=True
        )

        # ax is an Axes object → get its Figure
        fig = ax.get_figure()
        style_colorbar(decimals=scale_digits)
        # Remove margins
        fig.subplots_adjust(left=0, right=0.8, bottom=0, top=1)

        out_path = os.path.join(plot_save_dir, f'{dataset_name}_{name}_flatmap.png')
        plt.savefig(out_path, dpi=300)
        plt.close()
        print(f"[SAVE] Flatmap saved: {out_path}")


In [64]:
r2_age = data_r2_ordered.iloc[:1]
r2_sex = data_r2_ordered.iloc[1:2]

if dataset_name == "hcpd":
    r2_sc = data_r2_ordered.iloc[2:6]
    r2_scales = data_r2_ordered.iloc[6:]

elif dataset_name == "hcpa":
    r2_sc = data_r2_ordered.iloc[2:5]
    r2_scales = data_r2_ordered.iloc[5:]

In [65]:
r2_sc

region,M1L,M2L,M3L,M4L,A1L,A2L,A3L,D1L,D2L,D3L,...,A3R,D1R,D2R,D3R,D4R,S1R,S2R,S3R,S4R,S5R
r2_srsmoo,0,0,0.0,0.001,0,0.002,0.002,0,0.0,0.013,...,0,0.004,0.003,0.001,0.007,0.009,0.003,0.007,0.002,0
r2_srage1,0,0,0,0.0,0,0.002,0.001,0,0.001,0.011,...,0,0.004,0.004,0.0,0.005,0.009,0.005,0.007,0.003,0
r2_srage2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.004,0.0,0.001,0.004,0.001,0.001,0.002,0.0,0.0,0.002
r2_srsex,0,0.0,0.003,0.014,0,0.005,0.001,0,0,0.013,...,0,0.004,0.011,0.016,0.021,0.017,0.009,0.014,0.011,0


In [47]:
make_flatmaps(df = r2_age, nii_save_dir = f"{plot_dir}/niff", plot_save_dir = plot_dir, 
              symmetric = False, global_range=True, scale_digits = 3, cmap = 'YlOrBr_r', vmin=0, vmax=0.111)

[INFO] Global color scale: min=0.0, max=0.1
[INFO] r2_age: min=0, max=0.111
[SAVE] Flatmap saved: /Users/ruohan/My_Repos/Cerebellum/dat/hcpa/from_gam/plots/hcpa_r2_age_flatmap.png


In [66]:
make_flatmaps(df = r2_sex, nii_save_dir = f"{plot_dir}/niff", plot_save_dir = plot_dir, 
              symmetric = False, global_range=True, scale_digits = 3, cmap = 'YlOrBr_r', vmin=0, vmax=0.056)

[INFO] Global color scale: min=0.0, max=0.015
[INFO] r2_sex: min=0, max=0.056
[SAVE] Flatmap saved: /Users/ruohan/My_Repos/Cerebellum/dat/hcpd/from_gam/plots/hcpd_r2_sex_flatmap.png


In [49]:
make_flatmaps(df = r2_sc, nii_save_dir = f"{plot_dir}/niff", plot_save_dir = plot_dir, 
              symmetric = False, global_range=True, scale_digits = 3, cmap = 'YlOrBr_r', vmin=0, vmax=0.021)

[INFO] Global color scale: min=0.0, max=0.008
[INFO] r2_srsmoo: min=0, max=0.021
[SAVE] Flatmap saved: /Users/ruohan/My_Repos/Cerebellum/dat/hcpa/from_gam/plots/hcpa_r2_srsmoo_flatmap.png
[INFO] r2_srage1: min=0, max=0.021
[SAVE] Flatmap saved: /Users/ruohan/My_Repos/Cerebellum/dat/hcpa/from_gam/plots/hcpa_r2_srage1_flatmap.png
[INFO] r2_srsex: min=0, max=0.021
[SAVE] Flatmap saved: /Users/ruohan/My_Repos/Cerebellum/dat/hcpa/from_gam/plots/hcpa_r2_srsex_flatmap.png


# Interpretation of SC in HCP-D

The hierarchical models’ ΔR² values suggest that general social connectedness (SC) accounted for additional variance in cerebellar morphology within the D3 and S1 regions. While initial effects were observed in both regions prior to multiple-comparison correction, these did not remain significant following FDR adjustment.

After accounting for the general effect, there appear to be sex-dependent associations. Biological male and female individuals showed different relationships between sc scores and cerebellar morphology. These differences are most pronounced in the right social and demand regions.


In [51]:
make_flatmaps(df = r2_scales, nii_save_dir = f"{plot_dir}/niff", plot_save_dir = plot_dir, 
              symmetric = False, global_range=True, scale_digits = 3, cmap = 'YlOrBr_r', vmin=0, vmax=0.025)

[INFO] Global color scale: min=0.0, max=0.015
[INFO] r2_lonsmoo: min=0, max=0.025
[SAVE] Flatmap saved: /Users/ruohan/My_Repos/Cerebellum/dat/hcpa/from_gam/plots/hcpa_r2_lonsmoo_flatmap.png
[INFO] r2_lonage1: min=0, max=0.025
[SAVE] Flatmap saved: /Users/ruohan/My_Repos/Cerebellum/dat/hcpa/from_gam/plots/hcpa_r2_lonage1_flatmap.png
[INFO] r2_lonsex: min=0, max=0.025
[SAVE] Flatmap saved: /Users/ruohan/My_Repos/Cerebellum/dat/hcpa/from_gam/plots/hcpa_r2_lonsex_flatmap.png
[INFO] r2_rejsmoo: min=0, max=0.025
[SAVE] Flatmap saved: /Users/ruohan/My_Repos/Cerebellum/dat/hcpa/from_gam/plots/hcpa_r2_rejsmoo_flatmap.png
[INFO] r2_rejage1: min=0, max=0.025
[SAVE] Flatmap saved: /Users/ruohan/My_Repos/Cerebellum/dat/hcpa/from_gam/plots/hcpa_r2_rejage1_flatmap.png
[INFO] r2_rejsex: min=0, max=0.025
[SAVE] Flatmap saved: /Users/ruohan/My_Repos/Cerebellum/dat/hcpa/from_gam/plots/hcpa_r2_rejsex_flatmap.png
[INFO] r2_hossmoo: min=0, max=0.025
[SAVE] Flatmap saved: /Users/ruohan/My_Repos/Cerebellum/da

# Plot mean derivatives
- load data and tranpose to rowwise for plotting
- set the negative R2 to 0
- load the cerebeller altlas and reorder the data accordingly
- check data range to decide range for color bar between plots
- function for plotting
- plot and save the .nii files and flatmap plots

In [52]:
fusion_deriv = pd.read_csv(os.path.join(data_dir, dataset_name, 'from_gam', f"{dataset_name}_mean_deriv_fusion.csv"))
fusion_deriv.columns = fusion_deriv.columns.str.replace(".", "_", regex=False)

print(fusion_deriv.columns)

Index(['region', 'sr_mod_srsmoo_deriv_ssr', 'sr_mod_srsmoo_deriv_sicv',
       'sr_mod_srsmoo_deriv_sage', 'sr_mod_srage1',
       'sr_mod_srsex_deriv_ssrsexF', 'sr_mod_srsex_deriv_ssrsexM',
       'lon_mod_lonsmoo', 'lon_mod_lonage1',
       'lon_mod_lonsex_deriv_srawlonelisexF',
       'lon_mod_lonsex_deriv_srawlonelisexM', 'rej_mod_rejsmoo',
       'rej_mod_rejage1', 'rej_mod_rejsex_deriv_srawrejsexF',
       'rej_mod_rejsex_deriv_srawrejsexM', 'hos_mod_hossmoo',
       'hos_mod_hosage1', 'hos_mod_hossex_deriv_srawhostilitysexF',
       'hos_mod_hossex_deriv_srawhostilitysexM', 'emo_mod_emosmoo',
       'emo_mod_emoage1', 'emo_mod_emosex_deriv_srawemosupsexF',
       'emo_mod_emosex_deriv_srawemosupsexM', 'fri_mod_frismoo',
       'fri_mod_friage1', 'fri_mod_frisex_deriv_srawfriendssexF',
       'fri_mod_frisex_deriv_srawfriendssexM'],
      dtype='object')


In [53]:
# transpose the data to plot per row
fusion_deriv = fusion_deriv.T
fusion_deriv.columns = fusion_deriv.iloc[0]
fusion_deriv = fusion_deriv[1:]
fusion_deriv.head()

region,A1L,A1R,A2L,A2R,A3L,A3R,D1L,D1R,D2L,D2R,...,S1L,S1R,S2L,S2R,S3L,S3R,S4L,S4R,S5L,S5R
sr_mod_srsmoo_deriv_ssr,0.65,1.724,9.248,4.479,1.227,-0.11,1.193,1.055,14.243,13.896,...,7.298,0.548,4.044,11.838,-1.556,1.339,0.992,4.788,0.158,1.359
sr_mod_srsmoo_deriv_sicv,0.0,0.0,0.0,0.001,0.0,0.0,0.001,0.001,0.002,0.002,...,0.001,0.001,0.002,0.002,0.0,0.0,0.001,0.001,0.0,0.0
sr_mod_srsmoo_deriv_sage,-0.326,-0.909,-2.285,-5.133,-0.46,-0.934,2.498,-0.879,-8.784,-2.746,...,7.24,-2.524,-1.383,-5.698,-0.007,-0.144,-3.106,-1.093,-0.978,-1.584
sr_mod_srage1,0.708,2.162,8.888,6.18,1.524,0.496,4.623,1.898,22.318,18.472,...,15.499,4.323,21.24,20.174,-1.518,1.589,4.554,6.841,0.268,1.725
sr_mod_srsex_deriv_ssrsexF,5.516,-1.317,13.634,3.931,6.14,-0.706,19.859,2.476,8.679,15.734,...,64.52,-7.551,32.214,5.111,-1.063,1.489,-6.592,8.405,2.7,0.912


In [54]:
# Reorder data columns to match the order defined in lab_num
data_deriv_ordered = fusion_deriv[lab_num['parcel_name'].tolist()]
print(data_deriv_ordered.columns)

Index(['M1L', 'M2L', 'M3L', 'M4L', 'A1L', 'A2L', 'A3L', 'D1L', 'D2L', 'D3L',
       'D4L', 'S1L', 'S2L', 'S3L', 'S4L', 'S5L', 'M1R', 'M2R', 'M3R', 'M4R',
       'A1R', 'A2R', 'A3R', 'D1R', 'D2R', 'D3R', 'D4R', 'S1R', 'S2R', 'S3R',
       'S4R', 'S5R'],
      dtype='object', name='region')


In [55]:
row_min = data_deriv_ordered.min(axis=1)
row_max = data_deriv_ordered.max(axis=1)
row_range = pd.DataFrame({
    'min': row_min,
    'max': row_max
})
row_range

,min,max
sr_mod_srsmoo_deriv_ssr,-10.202,35.017
sr_mod_srsmoo_deriv_sicv,0.0,0.003
sr_mod_srsmoo_deriv_sage,-16.493,7.24
sr_mod_srage1,-5.836,38.185
sr_mod_srsex_deriv_ssrsexF,-35.211,64.52
sr_mod_srsex_deriv_ssrsexM,-44.357,45.56
lon_mod_lonsmoo,-11.244,10.44
lon_mod_lonage1,-12.908,9.255
lon_mod_lonsex_deriv_srawlonelisexF,-26.639,16.299
lon_mod_lonsex_deriv_srawlonelisexM,-9.594,18.264


In [56]:
deriv_age = data_deriv_ordered.iloc[2:3] # skip icv
if dataset_name == "hcpd":
    deriv_sc = data_deriv_ordered.iloc[[0,3,4,5,6]]
    deriv_scales = data_deriv_ordered.iloc[7:]

elif dataset_name == "hcpa":
    deriv_sc = data_deriv_ordered.iloc[[0,3,4,5]]
    deriv_scales = data_deriv_ordered.iloc[6:]

In [57]:
deriv_sc

region,M1L,M2L,M3L,M4L,A1L,A2L,A3L,D1L,D2L,D3L,...,A3R,D1R,D2R,D3R,D4R,S1R,S2R,S3R,S4R,S5R
sr_mod_srsmoo_deriv_ssr,-4.785,-9.502,15.328,-1.76,0.65,9.248,1.227,1.193,14.243,35.017,...,-0.11,1.055,13.896,3.211,2.508,0.548,11.838,1.339,4.788,1.359
sr_mod_srage1,-4.471,-3.239,18.923,-4.866,0.708,8.888,1.524,4.623,22.318,38.185,...,0.496,1.898,18.472,8.776,4.24,4.323,20.174,1.589,6.841,1.725
sr_mod_srsex_deriv_ssrsexF,-1.689,16.118,47.715,10.775,5.516,13.634,6.14,19.859,8.679,5.943,...,-0.706,2.476,15.734,12.113,7.223,-7.551,5.111,1.489,8.405,0.912
sr_mod_srsex_deriv_ssrsexM,-2.15,-11.289,2.251,-35.624,-5.758,4.119,-19.093,-2.19,16.646,45.56,...,0.682,-0.724,11.424,-32.306,-6.764,11.268,20.743,1.145,-1.779,1.824


In [59]:
make_flatmaps(df = deriv_age, nii_save_dir = f"{plot_dir}/niff", plot_save_dir = plot_dir, 
              symmetric = True, global_range=True, scale_digits = 0, cmap = 'coolwarm', vmin=-17, vmax=17)

[INFO] Global color scale: min=-16.493, max=16.493
[INFO] sr_mod_srsmoo_deriv_sage: min=-17, max=17
[SAVE] Flatmap saved: /Users/ruohan/My_Repos/Cerebellum/dat/hcpa/from_gam/plots/hcpa_sr_mod_srsmoo_deriv_sage_flatmap.png


In [60]:
make_flatmaps(df = deriv_sc, nii_save_dir = f"{plot_dir}/niff", plot_save_dir = plot_dir, 
              symmetric = True, global_range=True, scale_digits = 0, cmap = 'coolwarm', vmin=-372, vmax=372)

[INFO] Global color scale: min=-64.52, max=64.52
[INFO] sr_mod_srsmoo_deriv_ssr: min=-372, max=372
[SAVE] Flatmap saved: /Users/ruohan/My_Repos/Cerebellum/dat/hcpa/from_gam/plots/hcpa_sr_mod_srsmoo_deriv_ssr_flatmap.png
[INFO] sr_mod_srage1: min=-372, max=372
[SAVE] Flatmap saved: /Users/ruohan/My_Repos/Cerebellum/dat/hcpa/from_gam/plots/hcpa_sr_mod_srage1_flatmap.png
[INFO] sr_mod_srsex_deriv_ssrsexF: min=-372, max=372
[SAVE] Flatmap saved: /Users/ruohan/My_Repos/Cerebellum/dat/hcpa/from_gam/plots/hcpa_sr_mod_srsex_deriv_ssrsexF_flatmap.png
[INFO] sr_mod_srsex_deriv_ssrsexM: min=-372, max=372
[SAVE] Flatmap saved: /Users/ruohan/My_Repos/Cerebellum/dat/hcpa/from_gam/plots/hcpa_sr_mod_srsex_deriv_ssrsexM_flatmap.png


In [61]:
make_flatmaps(df = deriv_scales, nii_save_dir = f"{plot_dir}/niff", plot_save_dir = plot_dir, 
              symmetric = True, global_range=True, scale_digits = 0, cmap = 'coolwarm', vmin=-53, vmax=53)

[INFO] Global color scale: min=-47.886, max=47.886
[INFO] lon_mod_lonsmoo: min=-53, max=53
[SAVE] Flatmap saved: /Users/ruohan/My_Repos/Cerebellum/dat/hcpa/from_gam/plots/hcpa_lon_mod_lonsmoo_flatmap.png
[INFO] lon_mod_lonage1: min=-53, max=53
[SAVE] Flatmap saved: /Users/ruohan/My_Repos/Cerebellum/dat/hcpa/from_gam/plots/hcpa_lon_mod_lonage1_flatmap.png
[INFO] lon_mod_lonsex_deriv_srawlonelisexF: min=-53, max=53
[SAVE] Flatmap saved: /Users/ruohan/My_Repos/Cerebellum/dat/hcpa/from_gam/plots/hcpa_lon_mod_lonsex_deriv_srawlonelisexF_flatmap.png
[INFO] lon_mod_lonsex_deriv_srawlonelisexM: min=-53, max=53
[SAVE] Flatmap saved: /Users/ruohan/My_Repos/Cerebellum/dat/hcpa/from_gam/plots/hcpa_lon_mod_lonsex_deriv_srawlonelisexM_flatmap.png
[INFO] rej_mod_rejsmoo: min=-53, max=53
[SAVE] Flatmap saved: /Users/ruohan/My_Repos/Cerebellum/dat/hcpa/from_gam/plots/hcpa_rej_mod_rejsmoo_flatmap.png
[INFO] rej_mod_rejage1: min=-53, max=53
[SAVE] Flatmap saved: /Users/ruohan/My_Repos/Cerebellum/dat/hcpa

In [47]:
fusion_sex = pd.read_csv(os.path.join(data_dir, dataset_name, 'from_gam', f"{dataset_name}_GAM_sex_stats_fusion.csv"))
print(fusion_sex.columns)

Index(['mean_F', 'mean_M', 'SE_F', 'SE_M', 'contrast', 'estimate', 'SE', 'df',
       't.ratio', 'p.value', 'cohen_d', 'model', 'p.fdr', 'sig'],
      dtype='object')


In [48]:
fusion_sex = fusion_sex.set_index("model")
fusion_sex = fusion_sex.T
data_sex_d = fusion_sex.loc[["cohen_d"]]

In [49]:
data_sex_d

model,A1L,A1R,A2L,A2R,A3L,A3R,D1L,D1R,D2L,D2R,...,S1L,S1R,S2L,S2R,S3L,S3R,S4L,S4R,S5L,S5R
cohen_d,-0.729,-0.315,-0.499,-0.242,-0.691,-0.205,-0.468,-0.37,-0.363,-0.396,...,-0.513,-0.231,-0.328,-0.278,-0.476,-0.226,-0.568,-0.401,-0.471,-0.247


In [50]:
# Reorder data columns to match the order defined in lab_num
data_sex_ordered = data_sex_d[lab_num['parcel_name'].tolist()]
print(data_sex_ordered.columns)

Index(['M1L', 'M2L', 'M3L', 'M4L', 'A1L', 'A2L', 'A3L', 'D1L', 'D2L', 'D3L',
       'D4L', 'S1L', 'S2L', 'S3L', 'S4L', 'S5L', 'M1R', 'M2R', 'M3R', 'M4R',
       'A1R', 'A2R', 'A3R', 'D1R', 'D2R', 'D3R', 'D4R', 'S1R', 'S2R', 'S3R',
       'S4R', 'S5R'],
      dtype='object', name='model')


In [51]:
row_min = data_sex_ordered.min(axis=1)
row_max = data_sex_ordered.max(axis=1)
row_range = pd.DataFrame({
    'min': row_min,
    'max': row_max
})
row_range

,min,max
cohen_d,-0.729,-0.097


In [52]:
make_flatmaps(df = data_sex_ordered, nii_save_dir = f"{plot_dir}/niff", plot_save_dir = plot_dir, 
              symmetric = True, global_range=True, scale_digits = 3, cmap = 'PuOr_r', vmin=-0.730, vmax=0.730)

[INFO] Global color scale: min=-0.729, max=0.729
[INFO] cohen_d: min=-0.73, max=0.73
[SAVE] Flatmap saved: /Users/ruohan/My_Repos/Cerebellum/dat/hcpa/from_gam/plots/hcpa_cohen_d_flatmap.png
